In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import re
import itertools
import time
import csv
from webdriver_manager.chrome import ChromeDriverManager # Use this instead of local path

ModuleNotFoundError: No module named 'webdriver_manager'

## Get different categories page

In [2]:
# Function to extract category URLs
def get_cat(full_url):
    drivermanager = ChromeDriverManager().install()
    # Selenium driver
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    #path = '/Users/williamlee/Desktop/Work/Changing_room/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.get(full_url)

    # time.sleep(t)

    # Find class for tags with URLs to loop through
    elems = driver.find_elements_by_tag_name('a')
    cat_list = [elem.get_attribute('href') for elem in elems]

    driver.quit()

    return cat_list

In [3]:
# Get all URLs and remove NoneType values
cat_list = get_cat('https://www.consciousthelabel.com')
cat_list = [string for string in cat_list if string]

<ipython-input-2-09892e4addf0>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [ ]:
# manually change: the substrings: need to go on the webpage to get the idea of that page
substring = ['/shop', '/apparel']
cat_list = [string for string in cat_list if any(sub in string for sub in substring)]

In [ ]:
cat_list

['https://www.consciousthelabel.com/shop',
 'https://www.consciousthelabel.com/apparel',
 'https://www.consciousthelabel.com/apparel',
 'https://www.consciousthelabel.com/shop',
 'https://www.consciousthelabel.com/apparel']

In [ ]:
cat_list = list(set(cat_list))

In [ ]:
cat_list

['https://www.consciousthelabel.com/shop',
 'https://www.consciousthelabel.com/apparel']

In [ ]:
# # The first driver does the same thing
# url = 'https://www.consciousthelabel.com'
# response = requests.get(url)
# if not response.status_code == 200:
#     print('HTTP error', response.status_code, f'with {key} at {url}')
#     headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
#     response = requests.get(url, headers=headers)
# else:
#     print('Resolved!')
# page = BeautifulSoup(response.content, 'lxml')
# a_tag = page.find_all('a')

# a_list = []
# for a in a_tag:
#     try:
#         string = a['href']
#         if any(sub in string for sub in substring):
#             a_list.append(string)
#     except KeyError:
#         pass
# a_list

## Get different product page

In [ ]:
# get all product link: couldn't scroll to the last page
# p_dict = {}
# substring = ['/product-page/']

# def get_product_link(url):
#     response = requests.get(url)
#     if not response.status_code == 200:
#         print('HTTP error', response.status_code, f'with {key} at {url}')
#         headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
#         response = requests.get(url, headers=headers)
#     else:
#         print('Resolved!')
#     page = BeautifulSoup(response.content, 'lxml')
#     a_tag = page.find_all('a')

#     for a in a_tag:
#         try:
#             string = a['href']
#             if any(sub in string for sub in substring):
#                 p_dict[a.getText()] = string
#         except KeyError:
#             pass
#     return p_dict

In [ ]:
# for i in range(len(cat_list)):
#     p_dict_load = get_product_link(cat_list[i])

Resolved!
Resolved!


In [ ]:
p_dict = {}
substring = ['/product-page/']

def cat_itemize(cat_url):

    # Selenium driver
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    path = '/Users/williamlee/Desktop/Work/Changing_room/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument("--window-size=1920,1080")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.get(cat_url)
    
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # Name of the category
    elems = driver.find_elements_by_tag_name('a')
    for elem in elems:
        string = elem.get_attribute('href')
        if any(sub in string for sub in substring):
            p_dict[elem.text] = string
    return p_dict


In [ ]:
for i in range(len(cat_list)):
    p_dict_load = cat_itemize(cat_list[i])

In [ ]:
len(p_dict_load)

84

## DataFrame

In [ ]:
conscious_df = pd.DataFrame(columns=['Name', 'Material', 'Color', 'Size', 'Price', 'URL', 'Image', 'Brand_name', 'Description'])
def conscious_scrpaer(p_dict):
    for product, url in tqdm(p_dict.items()):
        product_dict = {}
        product_dict['Name'] = product.split('\n')[0]
        try:
            product_dict['Price'] = product.split('\n')[2]
        except:
            pass
        product_dict['URL'] = url
        product_dict['Brand_name'] = 'conscious-swimwear'

        response = requests.get(url)
        if not response.status_code == 200:
            print('HTTP error', response.status_code, f'with {key} at {url}')
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
            response = requests.get(url, headers=headers)

        page = BeautifulSoup(response.content, 'lxml')
        descriptions = page.find('pre', {'data-hook':'description', 'class':'_28cEs'}).find_all('p')
        description = ''
        for des in descriptions:
            if not des.get_text().startswith('Model'):
                description = description + des.get_text() + ' '
        product_dict['Description'] = description

        pattern =re.compile('[\d]+% [A-Za-z ]+cotton|[\d]+% [A-Za-z ]+plastic|[\d]+%[A-Za-z ]+elastane')
        material = re.findall(pattern, description)
        product_dict['Material'] = material


        image = page.find('div', {'id':'get-image-item-id'})['href']
        product_dict['Image'] = image

        #sizes is in a button, need to use selenium to get it
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
        path = '/Users/williamlee/Desktop/Work/Changing_room/chromedriver'
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument("--window-size=1920,1080")
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--allow-running-insecure-content')
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(executable_path=path, options=options)
        driver.get(url)
        button = driver.find_element_by_xpath("//div[@data-hook='popover-element']//button[@type='button']")
        button.click()
        elems = driver.find_elements_by_xpath("//div[@data-hook='option']")
        sizes = [elem.text for elem in elems]
        product_dict['Size'] = sizes

        conscious_df = conscious_df.append(product_dict, ignore_index=True)






  0%|          | 0/84 [00:00<?, ?it/s]




  1%|          | 1/84 [00:09<13:16,  9.59s/it]




  2%|▏         | 2/84 [00:15<11:29,  8.41s/it]




  4%|▎         | 3/84 [00:20<10:13,  7.58s/it]




  5%|▍         | 4/84 [00:26<09:11,  6.89s/it]




  6%|▌         | 5/84 [00:33<09:05,  6.91s/it]




  7%|▋         | 6/84 [00:47<11:42,  9.00s/it]




  8%|▊         | 7/84 [00:54<10:59,  8.56s/it]




 10%|▉         | 8/84 [00:58<09:14,  7.30s/it]




 11%|█         | 9/84 [01:04<08:18,  6.65s/it]




 12%|█▏        | 10/84 [01:09<07:54,  6.41s/it]




 13%|█▎        | 11/84 [01:16<07:44,  6.36s/it]




 14%|█▍        | 12/84 [01:22<07:35,  6.32s/it]




 15%|█▌        | 13/84 [01:30<08:04,  6.82s/it]




 17%|█▋        | 14/84 [01:36<07:34,  6.50s/it]




 18%|█▊        | 15/84 [01:40<06:44,  5.86s/it]




 19%|█▉        | 16/84 [01:45<06:19,  5.58s/it]




 20%|██        | 17/84 [01:52<06:43,  6.02s/it]




 21%|██▏       | 18/84 [01:58<06:45,  6.14s/it]




 23%|██▎       | 19/84 [0

In [ ]:
conscious_scrpaer(p_dict)

In [ ]:
conscious_df.to_csv('conscious_version1.csv', index=False)